In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
df = pd.read_csv('text_emotion.csv')
df.head()
df = df[pd.notnull(df['content'])]
col=['sentiment','content']
df=df[col]
df.columns
df.columns=['sentiment','content']

In [3]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

df['content'] = np.array([ clean_tweet(tweet) for tweet in df['content'] ])
print(df.head(10))


    sentiment                                            content
0       empty  i know i was listenin to bad habit earlier and...
1     sadness  Layin n bed with a headache ughhhh waitin on y...
2     sadness                     Funeral ceremony gloomy friday
3  enthusiasm                wants to hang out with friends SOON
4     neutral  We want to trade with someone who has Houston ...
5       worry  Re pinging why didn t you go to prom BC my bf ...
6     sadness  I should be sleep but im not thinking about an...
7       worry                                       Hmmm is down
8     sadness                        Charlene my love I miss you
9     sadness                     I m sorry at least it s Friday


In [5]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['content'].values)
X = tokenizer.texts_to_sequences(df['content'].values)
X = pad_sequences(X)

In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 36, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 36, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 13)                2561      
Total params: 513,361
Trainable params: 513,361
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(26800, 36) (26800, 13)
(13200, 36) (13200, 13)


In [11]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 106s - loss: 2.0457 - acc: 0.2913
Epoch 2/7
 - 109s - loss: 1.8854 - acc: 0.3610
Epoch 3/7
 - 120s - loss: 1.8240 - acc: 0.3806
Epoch 4/7
 - 104s - loss: 1.7780 - acc: 0.3981
Epoch 5/7
 - 99s - loss: 1.7406 - acc: 0.4088
Epoch 6/7
 - 100s - loss: 1.6967 - acc: 0.4183
Epoch 7/7
 - 98s - loss: 1.6640 - acc: 0.4323


In [12]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.94
acc: 0.35
